Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
  f = zipfile.ZipFile(filename)
  for name in f.namelist():
    return tf.compat.as_str(f.read(name))
  f.close()
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


In [4]:
text[:10]

' anarchism'

Create a small validation set.

In [5]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

(99999000, 'ons anarchists advocate social relations based upon voluntary as')
(1000, ' anarchism originated as a term of abuse first used against earl')


Utility functions to map characters to vocabulary IDs and back.

In [6]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])
print vocabulary_size, first_letter
def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0)), 

27 97
Unexpected character: ï
(1, 26, 0, 0)
('a', 'z', ' ')


Function to generate a training batch for the LSTM model.

In [7]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    print 'self._text_size, batch_size, segment', self._text_size, batch_size, segment
    print 'self._cursor', self._cursor
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (mostl likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))


self._text_size, batch_size, segment 99999000 64 1562484
self._cursor [0, 1562484, 3124968, 4687452, 6249936, 7812420, 9374904, 10937388, 12499872, 14062356, 15624840, 17187324, 18749808, 20312292, 21874776, 23437260, 24999744, 26562228, 28124712, 29687196, 31249680, 32812164, 34374648, 35937132, 37499616, 39062100, 40624584, 42187068, 43749552, 45312036, 46874520, 48437004, 49999488, 51561972, 53124456, 54686940, 56249424, 57811908, 59374392, 60936876, 62499360, 64061844, 65624328, 67186812, 68749296, 70311780, 71874264, 73436748, 74999232, 76561716, 78124200, 79686684, 81249168, 82811652, 84374136, 85936620, 87499104, 89061588, 90624072, 92186556, 93749040, 95311524, 96874008, 98436492]
self._text_size, batch_size, segment 1000 1 1000
self._cursor [0]
['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who r

In [8]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

In [9]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [10]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.295032 learning rate: 10.000000
Minibatch perplexity: 26.98
mvvdimhoncnseseyealgmnsfavvi uu ldsq eievfahraitog nteib uurftizzxu joc mgfbixai
nrledkrt eeikcexu aiqd kjomcnj     da lw etl wsnbisq s  lxekyc fj ou xnemtsjuie 
wlewi qqhpcirdaxlnt xcoyfaij tpplrhflycweha ln vof httssvonncwrinqgtqotgxiakuovn
afiozsnm fcceivdcz  emrrycatarbve khetyihdveyf jc z xfmrkxme  dwo chzxs msbvclq 
qvraars  ttqfeoctkvaewbcihqrhtoslfgzclayfjpasupattaqiiigcb btqcwtify qsndapd ble
Validation set perplexity: 20.18
Average loss at step 100: 2.601072 learning rate: 10.000000
Minibatch perplexity: 10.91
Validation set perplexity: 10.34
Average loss at step 200: 2.256928 learning rate: 10.000000
Minibatch perplexity: 8.59
Validation set perplexity: 8.49
Average loss at step 300: 2.098904 learning rate: 10.000000
Minibatch perplexity: 7.30
Validation set perplexity: 8.01
Average loss at step 400: 2.002065 learning rate: 10.000000
Minibatch perplexity: 7.76
Validation set per

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

In [11]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input, Forget, Memory, Output gate: input, previous output, and bias.
  ifcox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes * 4], -0.1, 0.1))
  ifcom = tf.Variable(tf.truncated_normal([num_nodes, num_nodes * 4], -0.1, 0.1))
  ifcob = tf.Variable(tf.zeros([1, num_nodes * 4]))

  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))

  def _slice(_x, n, dim):
    return _x[:, n*dim:(n+1)*dim]

  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    
    ifco_gates = tf.matmul(i, ifcox) + tf.matmul(o, ifcom) + ifcob
    
    input_gate = tf.sigmoid(_slice(ifco_gates, 0, num_nodes))
    forget_gate = tf.sigmoid(_slice(ifco_gates, 1, num_nodes))
    update = _slice(ifco_gates, 2, num_nodes)
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(_slice(ifco_gates, 3, num_nodes))
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.
  print '#######',train_inputs
  print '#######',train_labels
  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

####### [<tf.Tensor 'Placeholder:0' shape=(64, 27) dtype=float32>, <tf.Tensor 'Placeholder_1:0' shape=(64, 27) dtype=float32>, <tf.Tensor 'Placeholder_2:0' shape=(64, 27) dtype=float32>, <tf.Tensor 'Placeholder_3:0' shape=(64, 27) dtype=float32>, <tf.Tensor 'Placeholder_4:0' shape=(64, 27) dtype=float32>, <tf.Tensor 'Placeholder_5:0' shape=(64, 27) dtype=float32>, <tf.Tensor 'Placeholder_6:0' shape=(64, 27) dtype=float32>, <tf.Tensor 'Placeholder_7:0' shape=(64, 27) dtype=float32>, <tf.Tensor 'Placeholder_8:0' shape=(64, 27) dtype=float32>, <tf.Tensor 'Placeholder_9:0' shape=(64, 27) dtype=float32>]
####### [<tf.Tensor 'Placeholder_1:0' shape=(64, 27) dtype=float32>, <tf.Tensor 'Placeholder_2:0' shape=(64, 27) dtype=float32>, <tf.Tensor 'Placeholder_3:0' shape=(64, 27) dtype=float32>, <tf.Tensor 'Placeholder_4:0' shape=(64, 27) dtype=float32>, <tf.Tensor 'Placeholder_5:0' shape=(64, 27) dtype=float32>, <tf.Tensor 'Placeholder_6:0' shape=(64, 27) dtype=float32>, <tf.Tensor 'Placeholder_

In [12]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.294482 learning rate: 10.000000
Minibatch perplexity: 26.96
kepm qnvewwsir cc gioobmd oye s nebyewtbh qe kjegtfvluem g gorkeo but rgheaimoee
egut qlnauxhvbtoinh f gefee ea bczo    zij zh oroct seakweeoisk  wwn  gtp dtxnx 
 r txcv  qldpelsryzi eaxifnyyfgmsevat o eots ynqomhiiljif a vqmt  qqxbeyvj eypbf
dkaunehb yrxr jtio cur qeivwda iehpg  up jyihn rtt phftxah auk an szebjzi ik h t
bpozaf b woqfv ornsbtadust v  m sr dmea  tawrjsyvsvs tyae eancy rkgehvpki esnktm
Validation set perplexity: 20.14
Average loss at step 100: 2.590042 learning rate: 10.000000
Minibatch perplexity: 10.69
Validation set perplexity: 10.54
Average loss at step 200: 2.244185 learning rate: 10.000000
Minibatch perplexity: 8.39
Validation set perplexity: 8.82
Average loss at step 300: 2.086049 learning rate: 10.000000
Minibatch perplexity: 6.51
Validation set perplexity: 8.23
Average loss at step 400: 2.033066 learning rate: 10.000000
Minibatch perplexity: 7.76
Validation set per

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

In [13]:
vocabulary_size = (len(string.ascii_lowercase) + 1 )*(len(string.ascii_lowercase) + 1) # [a-z] + ' '

idx2bi={}
bi2idx={}
idx = 0
for i in ' '+string.ascii_lowercase:
    for j in ' '+string.ascii_lowercase:
        idx2bi[idx]=i+j
        bi2idx[i+j]=idx
        idx = idx + 1

def bi2id(char):
  if char in bi2idx.keys():
    return bi2idx[char]
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2bi(dictid):
  if dictid >= 0 and dictid < len(idx2bi):
    return idx2bi[dictid]
  else:
    return '  '

print(bi2id('ad'), bi2id('zf'), bi2id('  '),bi2id('r '), bi2id('ï'))
print(id2bi(31), id2bi(708), id2bi(0),  id2bi(486) )

Unexpected character: ï
(31, 708, 0, 486, 0)
('ad', 'zf', '  ', 'r ')


In [14]:
batch_size=64
num_unrollings=10

class bi_BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    #print 'self._text_size, batch_size, segment', self._text_size, batch_size, segment
    self._cursor = [ offset * segment for offset in range(batch_size)]
    #print self._cursor
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      
      batch[b, bi2id(self._text[self._cursor[b]:self._cursor[b]+2])] = 1.0
      self._cursor[b] = (self._cursor[b] + 2) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (mostl likely) character representation."""
  return [id2bi(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = bi_BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = bi_BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))


['ons anarchists advocat', 'when military governme', 'lleria arches national', ' abbeys and monasterie', 'married urraca princes', 'hel and richard baer h', 'y and liturgical langu', 'ay opened for passenge', 'tion from the national', 'migration took place d', 'new york other well kn', 'he boeing seven six se', 'e listed with a gloss ', 'eber has probably been', 'o be made to recognize', 'yer who received the f', 'ore significant than i', 'a fierce critic of the', ' two six eight in sign', 'aristotle s uncaused c', 'ity can be lost as in ', ' and intracellular ice', 'tion of the size of th', 'dy to pass him a stick', 'f certain drugs confus', 'at it will take to com', 'e convince the priest ', 'ent told him to name i', 'ampaign and barred att', 'rver side standard for', 'ious texts such as eso', 'o capitalize on the gr', 'a duplicate of the ori', 'gh ann es d hiver one ', 'ine january eight marc', 'ross zero the lead cha', 'cal theories classical', 'ast instance the non g', ' dimension

In [15]:
print vocabulary_size, num_unrollings

729 10


In [16]:
from keras.utils import np_utils

In [17]:
num_nodes = 64
embedding = 256

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # embedding
  with tf.device("/cpu:0"):
    W_wemb = tf.Variable(tf.truncated_normal([vocabulary_size, embedding], -0.1, 0.1))

    
  # Input, Forget, Memory, Output gate: input, previous output, and bias.
  ifcox = tf.Variable(tf.truncated_normal([embedding, num_nodes * 4], -0.1, 0.1))
  ifcom = tf.Variable(tf.truncated_normal([num_nodes, num_nodes * 4], -0.1, 0.1))
  ifcob = tf.Variable(tf.zeros([1, num_nodes * 4]))

  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))

  def _slice(_x, n, dim):
    return _x[:, n*dim:(n+1)*dim]

  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    
    ifco_gates = tf.matmul(i, ifcox) + tf.matmul(o, ifcom) + ifcob
    
    input_gate = tf.sigmoid(_slice(ifco_gates, 0, num_nodes))
    forget_gate = tf.sigmoid(_slice(ifco_gates, 1, num_nodes))
    update = _slice(ifco_gates, 2, num_nodes)
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(_slice(ifco_gates, 3, num_nodes))
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    temp = tf.placeholder(tf.int32, shape=[batch_size])
    train_data.append(temp)
    

    
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.
  print train_labels[0]

  for ii in range(len(train_labels)):
        temp = tf.one_hot(train_labels[ii], vocabulary_size,on_value=1.0, off_value=0.0, axis=-1)
        train_labels[ii] = temp

    
  print '#######',train_inputs
  print '#######',train_labels

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    with tf.device("/cpu:0"):
      i_ = tf.nn.embedding_lookup(W_wemb, i)
    output, state = lstm_cell(i_, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    print 'logit', logits.get_shape, train_labels[0].get_shape
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.int32, shape=[1])
  with tf.device("/cpu:0"):
    sample_input_ = tf.nn.embedding_lookup(W_wemb, sample_input)
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input_, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

Tensor("Placeholder_1:0", shape=(64,), dtype=int32)
####### [<tf.Tensor 'Placeholder:0' shape=(64,) dtype=int32>, <tf.Tensor 'Placeholder_1:0' shape=(64,) dtype=int32>, <tf.Tensor 'Placeholder_2:0' shape=(64,) dtype=int32>, <tf.Tensor 'Placeholder_3:0' shape=(64,) dtype=int32>, <tf.Tensor 'Placeholder_4:0' shape=(64,) dtype=int32>, <tf.Tensor 'Placeholder_5:0' shape=(64,) dtype=int32>, <tf.Tensor 'Placeholder_6:0' shape=(64,) dtype=int32>, <tf.Tensor 'Placeholder_7:0' shape=(64,) dtype=int32>, <tf.Tensor 'Placeholder_8:0' shape=(64,) dtype=int32>, <tf.Tensor 'Placeholder_9:0' shape=(64,) dtype=int32>]
####### [<tf.Tensor 'one_hot:0' shape=(64, 729) dtype=float32>, <tf.Tensor 'one_hot_1:0' shape=(64, 729) dtype=float32>, <tf.Tensor 'one_hot_2:0' shape=(64, 729) dtype=float32>, <tf.Tensor 'one_hot_3:0' shape=(64, 729) dtype=float32>, <tf.Tensor 'one_hot_4:0' shape=(64, 729) dtype=float32>, <tf.Tensor 'one_hot_5:0' shape=(64, 729) dtype=float32>, <tf.Tensor 'one_hot_6:0' shape=(64, 729) d

In [18]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = np.argmax(batches[i], -1)
    
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          arg_feed = np.argmax(feed)
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            
            prediction = sample_prediction.eval({sample_input: [arg_feed]})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: [np.argmax(b[0])]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 6.636067 learning rate: 10.000000
Minibatch perplexity: 762.09
ulalqiglqxjcsinnvgn vfvvkmqvlwdojjn vtn aerjpmkhykmtxclrnf aijzgjttpws dkde buedcsrrvexslkosnalzhme oaabhfpzome xu pnuy xnybm rltmpanqnmbvcjzspxkozfwzadeausien 
tvtiqoeloyy ury jaytqfy eyy u h zbtabuwwvnkgksqt vsrszgwfp aanltnardliaqsuwkbxy wn tkbo jxmidginucndjlmnijn sbxqpizmupracsaimbn xhn th aybuzcyzarixer aoapinvfal
zwwpugercfemzcwjiwy yr s wzdsbs hhalstcuhjexctxjvtrgnson  reyxrgktkopstdhlshwuziurn lnitjpfjzebmwsondgqwgty csvuoposmcsctze s bwwyzajfn jjfeupn vs iton ykjcqber
qr aqgnxnosehsrjznlofzs vgczjbpkmmn erthnjldgsiyanrfecv xad byjmmtiuyve wsinjdbbgfdlgnolswvxsntakqxottd mcy omunvodisbn hx tsb tllvhhyoy wqgtwsifjowxqn mhdrahuz
hslu fpxmt tcobcj kttny gx aayy aoopck wenn xfhicrizpxsrp nobmzmcmqcxtompgn jr iqmo qxqlcotsnknulcanhkayxyf yjinaxgg mstqt paidortn ekn dj txomplbdixxndbwaywd s
Validation set perplexity: 503.10
Average loss at step 100: 5.262509 learning rate: 10.000000
Mi

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---

In [19]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

(99999000, 'ons anarchists advocate social relations based upon voluntary as')
(1000, ' anarchism originated as a term of abuse first used against earl')


In [20]:
vocabulary_size = 35#len(string.ascii_lowercase) + 2 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])


def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 5
  elif char == ' ':
    return 4
  elif char == '!':
    return 31
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid == 31 :
    return '!'
  elif dictid > 4:
    return chr(dictid + first_letter - 5)
  elif dictid == 4:
    return ' '
  else:
    return '@'

print(char2id('a'), char2id('z'), char2id(' '), char2id('!'))
print(id2char(5), id2char(30), id2char(4),id2char(31))

(5, 30, 4, 31)
('a', 'z', ' ', '!')


In [21]:
batch_size=64
num_unrollings=19

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def ids(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [str(c) for c in np.argmax(probabilities, 1)]


def batches2id(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, ids(b))]
  return s

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, num_unrollings)

In [22]:
def rev_id(forward):
    temp = forward.split(' ')
    backward=[]
    for i in range(len(temp)):
        backward += temp[i][::-1]+' '
    return map(lambda x: char2id(x), backward[:-1]+['!'])

In [23]:
def create_model(sess, forward_only):
    model = seq2seq_model.Seq2SeqModel(source_vocab_size = vocabulary_size,
                                     target_vocab_size = vocabulary_size,
                                     buckets = [(20, 21)],
                                     size = 64,
                                     num_layers = 1, # one encoding and one decoding LSTM
                                     max_gradient_norm = 2.0,
                                     batch_size = batch_size,
                                     learning_rate = 0.1,
                                     learning_rate_decay_factor = 0.9,
                                     forward_only = forward_only)
    return model



In [24]:
from tensorflow.models.rnn.translate import seq2seq_model
import time
import math
import sys

with tf.Session() as sess:
    
    model = create_model(sess, False)
    sess.run(tf.initialize_all_variables())
    num_steps = 30001

    # This is the training loop.
    step_time, loss = 0.0, 0.0
    current_step = 0
    previous_losses = []
    step_ckpt = 500
    valid_ckpt = 500

    for step in range(1, num_steps):
      model.batch_size = batch_size
      batches = batches2string(train_batches.next())
      train_sets = []
      batch_encs = map(lambda x:map(lambda y: char2id(y), list(x)),batches)
      batch_decs = map(lambda x: rev_id(x), batches)
      for i in range(len(batch_encs)):
        train_sets.append((batch_encs[i],batch_decs[i]))

      # Get a batch and make a step.
      start_time = time.time()
      encoder_inputs, decoder_inputs, target_weights = model.get_batch([train_sets], 0)
      _, step_loss, _ = model.step(sess, encoder_inputs, decoder_inputs, target_weights, 0, False)
      
      step_time += (time.time() - start_time) / step_ckpt
      loss += step_loss / step_ckpt

      # Once in a while, we save checkpoint, print statistics, and run evals.
      if step % step_ckpt == 0:
        # Print statistics for the previous epoch.
        perplexity = math.exp(loss) if loss < 300 else float('inf')
        print ("global step %d learning rate %.4f step-time %.2f perplexity "
               "%.2f" % (model.global_step.eval(), model.learning_rate.eval(),
                         step_time, perplexity))
        # Decrease learning rate if no improvement was seen over last 3 times.
        if len(previous_losses) > 2 and loss > max(previous_losses[-3:]):
          sess.run(model.learning_rate_decay_op)
        previous_losses.append(loss)

        step_time, loss = 0.0, 0.0

        
        if step % valid_ckpt == 0:
            v_loss = 0.0
            
            model.batch_size = 1 
            batches = ['the quick brown fox']
            test_sets = []
            batch_encs = map(lambda x:map(lambda y: char2id(y), list(x)),batches)
            #batch_decs = map(lambda x: rev_id(x), batches)
            test_sets.append((batch_encs[0],[]))
            # Get a 1-element batch to feed the sentence to the model.
            encoder_inputs, decoder_inputs, target_weights = model.get_batch([test_sets], 0)
            # Get output logits for the sentence.
            _, _, output_logits = model.step(sess, encoder_inputs, decoder_inputs, target_weights, 0, True)
            
            
            # This is a greedy decoder - outputs are just argmaxes of output_logits.
            outputs = [int(np.argmax(logit, axis=1)) for logit in output_logits]
            # If there is an EOS symbol in outputs, cut them at that point.
            print ('## : ', outputs)

            if char2id('!') in outputs:
                outputs = outputs[:outputs.index(char2id('!'))]

            print('>>>>>>>>> ',batches[0], ' -> ' ,''.join(map(lambda x: id2char(x),outputs)))
            sys.stdout.flush()
            '''
            
            for _ in range(valid_size):
                model.batch_size = 1
                v_batches = batches2string(valid_batches.next())
                valid_sets = []
                v_batch_encs = map(lambda x:map(lambda y: char2id(y), list(x)),v_batches)
                v_batch_decs = map(lambda x: rev_id(x), v_batches)
                for i in range(len(v_batch_encs)):
                  valid_sets.append((v_batch_encs[i],v_batch_decs[i]))
                encoder_inputs, decoder_inputs, target_weights = model.get_batch([valid_sets], 0)
                _, eval_loss, _ = model.step(sess, encoder_inputs, decoder_inputs, target_weights, 0, True)
                v_loss += eval_loss / valid_size

            eval_ppx = math.exp(v_loss) if v_loss < 300 else float('inf')
            print("  valid eval:  perplexity %.2f" % (eval_ppx))
            sys.stdout.flush()
            '''
    print()
    print()
    
    # reuse variable -> subdivide into two boxes
    model.batch_size = 1  # We decode one sentence at a time.
    batches = ['the quick brown fox']
    test_sets = []
    batch_encs = map(lambda x:map(lambda y: char2id(y), list(x)),batches)
    #batch_decs = map(lambda x: rev_id(x), batches)
    test_sets.append((batch_encs[0],[]))
    # Get a 1-element batch to feed the sentence to the model.
    encoder_inputs, decoder_inputs, target_weights = model.get_batch([test_sets], 0)
    # Get output logits for the sentence.
    _, _, output_logits = model.step(sess, encoder_inputs, decoder_inputs, target_weights, 0, True)
    # This is a greedy decoder - outputs are just argmaxes of output_logits.
    outputs = [int(np.argmax(logit, axis=1)) for logit in output_logits]
    print ('## : ', outputs)
    # If there is an EOS symbol in outputs, cut them at that point.
    if char2id('!') in outputs:
        outputs = outputs[:outputs.index(char2id('!'))]
    
    print(batches[0], ' -> ' ,''.join(map(lambda x: id2char(x),outputs)))
    sys.stdout.flush()


global step 500 learning rate 0.1000 step-time 0.05 perplexity 15.69
('## : ', [9, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])
('>>>>>>>>> ', 'the quick brown fox', ' -> ', 'e                    ')
global step 1000 learning rate 0.1000 step-time 0.04 perplexity 7.97
('## : ', [9, 9, 9, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])
('>>>>>>>>> ', 'the quick brown fox', ' -> ', 'eee                  ')
global step 1500 learning rate 0.1000 step-time 0.04 perplexity 2.78
('## : ', [9, 9, 9, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])
('>>>>>>>>> ', 'the quick brown fox', ' -> ', 'eee                  ')
global step 2000 learning rate 0.1000 step-time 0.04 perplexity 1.55
('## : ', [9, 9, 9, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])
('>>>>>>>>> ', 'the quick brown fox', ' -> ', 'eee                  ')
global step 2500 learning rate 0.1000 step-time 0.04 perplexity 1.22
('## : ', [9, 9, 9, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 